### Грузим строки оставленные в Тест множестве

In [1]:
import joblib
import pandas as pd
from src.OLS import *

df = pd.read_csv('data/realty_data.csv', encoding='utf-8').drop(columns='period')
df['product_name'] = df.product_name.str.replace(' ', ' ')
rs_coord = (55.7539, 37.6208)

df['rs_dist'] = (df['lat'] - rs_coord[0]) ** 2 + (df['lon'] - rs_coord[1]) ** 2

reg_pipe = joblib.load('pipe.pkl')
clusterer = joblib.load('clusterer.pkl')
train_index = joblib.load('train_index.pkl')
preds = joblib.load('preds.pkl')

df = df[~df.index.isin(train_index)].reset_index(drop=True)
df

,product_name,price,postcode,address_name,lat,lon,object_type,total_square,rooms,floor,city,settlement,district,area,description,source,rs_dist
0,"3-комнатная, 137 м²",63000000,127473.0,"2-й Щемиловский переулок, 5а",55.778894,37.608844,Квартира,137.00,3.0,6.0,Москва,NaN,Тверской район,NaN,Просторная квартира свободной планировки с пан...,ЦИАН,0.000768
1,"3-комнатная, 76 м²",16004680,NaN,"ЖК Прокшино, 8 к4",55.594802,37.431264,Квартира,76.00,3.0,6.0,Москва,NaN,Сосенское поселение,NaN,"Apт.1684018. 0,01% - гибкая ипотека! Воспользу...",Яндекс.Недвижимость,0.061236
2,"1-комнатная, 24 м²",7841776,NaN,"ЖК Прокшино, 6 к2",55.594332,37.428099,Квартира,24.00,1.0,10.0,Москва,NaN,Сосенское поселение,NaN,Продается однокомнатная квартира № 381 в новос...,Новострой-М,0.062596
3,"1-комнатная, 34,8 м²",7200000,140016.0,"Дружбы, 11/26",55.701957,37.966424,Квартира,34.80,1.0,16.0,Люберцы,NaN,NaN,Самолёт м-н,Агентам просьба не беспокоить!\nПродаётся свет...,Домклик,0.122154
4,"3-комнатная, 154,7 м²",170663394,123242.0,"Дружинниковская, 15",55.759822,37.575351,Квартира,154.68,3.0,16.0,Москва,NaN,Пресненский район,NaN,Предлагается трехкомнатная квартира в новом жи...,Домклик,0.002101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19760,"2-комнатная, 56 м²",11655000,108811.0,"бульвар Андрея Тарковского, к15",55.625693,37.341506,Квартира,56.00,2.0,3.0,Москва,NaN,Внуковское поселение,NaN,"Квартира расположена в Новой Москве, ЖК «Расск...",Яндекс.Недвижимость,0.094442
19761,"3-комнатная, 75 м²",53000000,123104.0,"Богословский переулок, 8/15 ст1",55.762616,37.599727,Квартира,75.00,3.0,2.0,Москва,NaN,Пресненский район,NaN,"Красивый фасадный дом. Потолки 3,4 м., без бал...",ЦИАН,0.000520
19762,"2-комнатная, 47,7 м²",12999999,125481.0,"Свободы, 81 ст5",55.860262,37.442163,Квартира,47.70,2.0,4.0,Москва,NaN,Северное Тушино район,NaN,К продаже представлена 2-х комнатная квартира ...,Домклик,0.043224
19763,"1-комнатная, 33,5 м²",8500000,108823.0,"Рязановское шоссе, 31 к1",55.476485,37.530099,Квартира,33.50,1.0,13.0,Москва,NaN,Рязановское поселение,NaN,Номер лота: вт-0398917.\nВашему вниманию предл...,Домклик,0.085186


In [35]:
import numpy as np

df['price_pred'] = np.where(preds<0, 0, preds)
df['signed_error'] = df['price'] - df['price_pred']
df['error'] = np.abs(df['signed_error'])
df['error_perc'] = df['error'] / df['price'] * 100
df['error_quantile'] = pd.qcut(df['error'], 5, labels=False).astype(str)

In [36]:
to_plot = df[(df.price < df.price.quantile(.95)) & (df.error > df.error.quantile(.2))]

### Карта распределения цен

Ниже представлена карта предложений, которые модель *НЕ ВИДЕЛА* при обучении. Исключены точки с чрезмерно высокими ценами (более 80 миллионов, 5% наблюдений), а также выбросы предсказаний модели (2% предсказаний)

In [37]:
import plotly.express as px

fig = px.scatter_mapbox(to_plot,
                        lat="lat",
                        lon="lon",
                        height=600,
                        width=600,
                        color='price',
                        zoom=9,
                        mapbox_style='open-street-map')
fig.show()

## Распределение ошибок (модуль разницы предсказания и целевой цены)

В целом, 3/4 точек попадают в синию зону (ошибка менее 10 миллионов рублей), что является относительно хорошим результатом

In [38]:
import plotly.express as px

fig = px.scatter_mapbox(to_plot,
                        lat="lat",
                        lon="lon",
                        height=600,
                        width=600,
                        color='error',
                        zoom=9,
                        mapbox_style='open-street-map')
fig.show()

Типовая и средняя ошибка в предсказании - 6-8 миллионов рублей (в одну сторону), при средней цене от 10 до 20 миллионов

In [39]:
to_plot.error.describe()

count    1.484600e+04
mean     8.740406e+06
std      6.700757e+06
min      2.542599e+06
25%      4.714012e+06
50%      6.838564e+06
75%      1.020241e+07
max      1.067185e+08
Name: error, dtype: float64

В среднем модель ошибается на 50% от целевой цены

In [40]:
to_plot.error_perc.describe()

count    14846.000000
mean        56.348791
std         33.536651
min          3.757055
25%         29.718805
50%         49.624989
75%         79.598522
max        330.725434
Name: error_perc, dtype: float64

1 degree_lat + 1 degree_lon = 127.2km
beta = -6.71 * 1e+7
beta_km = beta / 127.2 = -530 * 1e+3

In [49]:
ols = joblib.load('pipe.pkl')
print(ols['regressor'].get_summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.762
Model:                            OLS   Adj. R-squared:                  0.762
Method:                 Least Squares   F-statistic:                 1.684e+04
Date:                Fri, 07 Apr 2023   Prob (F-statistic):               0.00
Time:                        17:30:29   Log-Likelihood:            -1.4315e+06
No. Observations:               79057   AIC:                         2.863e+06
Df Residuals:                   79041   BIC:                         2.863e+06
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
C(cluster)[0.0]  -4.138e+06   3.97e+05  

In [47]:
from sklearn.metrics import r2_score, mean_squared_error

print(f'out-of-sample R2: {r2_score(df.price, df.price_pred)}, RMSE: {mean_squared_error(df.price, df.price_pred) ** 0.5}')

out-of-sample R2: 0.7592422183426217, RMSE: 17198378.949009344


В целом, модель объясняет 76% вариативности данных, что на 76% пунктов выше чем наивное предсказание==средняя цена.
Модель состоит из двух частей - линейной регрессии (обученной по OLS), коэффиценты которой можно интепретировать так:
- С увеличением расстояния от Красной Площади на 1 км, цена падает на 530 тысячи рублей
- В среднем, один дополнительный квадратный метр стоит 580тыс рублей
- В среднем, выбрать квартиру на X+1 этаже, стоит дешевле на 160 - 12 * X тысяч рублей (5 этаж стоит дешевле 4 на 110 тыс рублей)
Над OLS моделью можно поработать, в частности над кластеризацией и заменить на более осмысленное разбиение по районам и использовать TSLS.
-
Также, в модель включен fine-tuned NLP модель на основе SBERT (обученная на ошибках OLS), однако ее вклад в предсказание минимален и нужно поработать над ней.
